In [4]:
import sys

!{sys.executable} -m pip install textblob
!{sys.executable} -m pip install textblob-fr
!{sys.executable} -m pip install polyglot
!{sys.executable} -m pip install pyicu
!{sys.executable} -m pip install morfessor
!{sys.executable} -m pip install pycld2


     |████████████████████████████████| 41.4MB 261kB/s eta 0:00:012   |▏                               | 245kB 1.5MB/s eta 0:00:29     |▎                               | 409kB 1.5MB/s eta 0:00:29     |▍                               | 552kB 1.5MB/s eta 0:00:28     |▋                               | 819kB 1.5MB/s eta 0:00:28     |███                             | 4.0MB 2.3MB/s eta 0:00:16��██▏                          | 6.7MB 773kB/s eta 0:00:45     |████████                        | 10.4MB 747kB/s eta 0:00:42     |█████████                       | 11.8MB 3.8MB/s eta 0:00:08     |█████████▏                      | 11.9MB 3.8MB/s eta 0:00:08     |█████████▎                      | 12.0MB 3.8MB/s eta 0:00:08     |█████████▍                      | 12.2MB 3.8MB/s eta 0:00:08     |█████████▋                      | 12.4MB 3.8MB/s eta 0:00:08     |█████████▉                      | 12.7MB 1.7MB/s eta 0:00:17     |█████████▉                      | 12.8MB 1.7MB/s eta 0:00:17     |██████████▏       

In [1]:
import os
import json
import logging
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import exc
from datetime import datetime
import pandas as pd
from textblob import TextBlob
from textblob_fr import PatternTagger, PatternAnalyzer
from polyglot.detect import Detector
import pycld2 as cld2

In [11]:
log = logging.getLogger(__name__)

class RawsParser():
    def __init__(self):
        self.database = os.environ['SHARED_DB_URI']
        self.Base = automap_base()
        self.engine = create_engine(self.database)
        self.Base.prepare(self.engine, reflect=True)
        self.Platforms = self.Base.classes.platforms
        self.Raws = self.Base.classes.raws

    def get_platforms(self):
        # reflect the tables
        session = Session(self.engine)

        platforms = session.query(self.Platforms, self.Platforms.id).all()
                
        session.close()

        return platforms

    def get_raws(self, platform):
        session = Session(self.engine)
        try:
            query = session.query(self.Raws).filter_by(platform_id=platform.id,status=None).limit(20).statement
            df = pd.read_sql(query, self.engine)
            session.close()
            
        except exc.SQLAlchemyError as e:
            print(e)
            session.close()
            log.error(e)
        return df
    
    def parser_raw_data(self): 
        platforms = self.get_platforms()
        print(platforms)

        for platform in platforms:
            log.info(platform)
            print(platform)
            df_raws = self.get_raws(platform)
            df_raws = df_raws.drop(['created_at'], axis=1)
            df = pd.DataFrame()
            if platform.id == 1 and df_raws.empty is False:
                log.info('Parsing twitter')
                df = self.twitter_data(df_raws)
            elif platform.id == 2 and df_raws.empty is False:
                log.info('Parsing instagram')
                df = self.instagram_data(df_raws)
            else:
                log.error('There\'s an unmanaged platform here, do something.')
        return df
        self.engine.dispose()
        
    def twitter_data(self, df_raws):
        df_update= pd.DataFrame(columns=['id','status'])
        df_update['id']=df_raws['id']
        df_raws = df_raws.rename(columns={"api_id": "twitter_id", "id": "raw_id"})
        df_raws["twitter_id"] = pd.to_numeric(df_raws["twitter_id"])
        df_flattened = df_raws.raw_data.apply(lambda x: self.parse_json_twitter(x))
        df_flattened["twitter_id"] = pd.to_numeric(df_flattened["twitter_id"])
        df_result = pd.merge(df_raws, df_flattened, on=['twitter_id'])
        df_result = df_result.drop(['raw_data','status','platform_id'], axis=1)
        print(list(df_result.columns))
        df_result.to_sql('tweets',con=self.engine, if_exists='append', index=False)
        self.update_raw_status(df_update)
        return df_result

    def parse_json_twitter(self, df):
        twitter_id = df['id']
        twitter_user_id = df['user']['id']
        twitter_text = df['text']
        twitter_user_name = df['user']['name']
        twitter_followers_count = df['user']['followers_count']
        twitter_url  =  df['id']

        blob_twitter_lang = Detector(twitter_text, quiet=True)
        twitter_lang = blob_twitter_lang.language.code
        twitter_sentiment = self.detect_sentiment(twitter_text, twitter_lang)
        twitter_topics  = None

        created_at = df['created_at']
        return pd.Series([twitter_id, 
                          twitter_user_id, 
                          twitter_text, 
                          twitter_user_name, 
                          twitter_followers_count,
                          twitter_lang,
                          twitter_sentiment,
                          twitter_topics,
                          twitter_url,
                          created_at], 
                         index=['twitter_id', 
                                'twitter_user_id', 
                                'twitter_text', 
                                'twitter_user_name', 
                                'twitter_followers_count', 
                                'twitter_lang',
                                'twitter_sentiment',
                                'twitter_topics',
                                'twitter_url',
                                'created_at']) 

    def instagram_data(self, df_raws):
        df_update= pd.DataFrame(columns = ['id','status'])
        df_update['id']=df_raws['id']
        df_raws = df_raws.rename(columns={"api_id": "instagram_id", "id": "raw_id"})
        df_raws["instagram_id"] = pd.to_numeric(df_raws["instagram_id"])
        df_flattened = df_raws.raw_data.apply(lambda x: self.parse_json_instagram(x))
        df_flattened["instagram_id"] = pd.to_numeric(df_flattened["instagram_id"])
        df_result = pd.merge(df_raws, df_flattened, on=['instagram_id'])
        df_result = df_result.drop(['raw_data','status','platform_id'], axis=1)
        df_result.to_sql('instagrams',con=self.engine, if_exists='append', index=False)
        self.update_raw_status(df_update)
        
        return df_result

    def parse_json_instagram(self, df):
        instagram_id = df['id']
        instagram_user_id = df['owner']['id']
        instagram_user_name = None
        instagram_followers_count = None
        instagram_media_url = df['display_url']
        
        instagram_text  = df['edge_media_to_caption']['edges'][0]['node']['text']
        
        blob_instagram_lang = Detector(instagram_text, quiet=True)
        instagram_lang = blob_instagram_lang.language.code

        instagram_sentiment = self.detect_sentiment(instagram_text, instagram_lang)
            
        instagram_topics  = None
        instagram_url  = df['shortcode']
        created_at = datetime.fromtimestamp(df['taken_at_timestamp'])

        return pd.Series([instagram_id,
                         instagram_user_id,
                         instagram_user_name,
                         instagram_followers_count,
                         instagram_media_url,
                         instagram_sentiment,
                         instagram_text ,
                         instagram_lang ,
                         instagram_topics ,
                         instagram_url ,
                         created_at], 
                         index=['instagram_id',
                                'instagram_user_id',
                                'instagram_user_name',
                                'instagram_followers_count',
                                'instagram_media_url',
                                'instagram_sentiment',
                                'instagram_text',
                                'instagram_lang',
                                'instagram_topics',
                                'instagram_url',
                                'created_at']) 
    @staticmethod
    def detect_sentiment(text, lang):
        if lang == 'fr':
            sentiment = TextBlob(text, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer()).sentiment
            polarity = sentiment[0]
        else: 
            sentiment = TextBlob(text).sentiment
            polarity = sentiment.polarity
        if polarity > 0.3:
            text_sentiment = 'Positive' 
        elif polarity < -0.3:
            text_sentiment = 'Negative' 
        else:
            text_sentiment = 'Neutral or Undefined' 
        return text_sentiment

    def update_raw_status(self, df_update):
        df_update['status'] = 'Parsed'
        print(df_update.to_dict(orient='records'))
        session = Session(self.engine)
        try:
            session.bulk_update_mappings(
              self.Raws,
              df_update.to_dict(orient='records')
            )
            session.commit()
            session.close()
            
        except exc.SQLAlchemyError as e:
            session.rollback()
            print(e)
            session.close()
            log.error(e)
        pass
    
    
    
parser = RawsParser()
posts = parser.parser_raw_data()
posts

[(<sqlalchemy.ext.automap.platforms object at 0x7f24c0308eb8>, 1), (<sqlalchemy.ext.automap.platforms object at 0x7f24c0308d68>, 2)]
(<sqlalchemy.ext.automap.platforms object at 0x7f24c0308eb8>, 1)


Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


['raw_id', 'brand_id', 'keyword_id', 'twitter_id', 'twitter_user_id', 'twitter_text', 'twitter_user_name', 'twitter_followers_count', 'twitter_lang', 'twitter_sentiment', 'twitter_topics', 'twitter_url', 'created_at']
[{'id': 45, 'status': 'Parsed'}, {'id': 49, 'status': 'Parsed'}, {'id': 51, 'status': 'Parsed'}, {'id': 114, 'status': 'Parsed'}, {'id': 146, 'status': 'Parsed'}, {'id': 147, 'status': 'Parsed'}, {'id': 148, 'status': 'Parsed'}, {'id': 149, 'status': 'Parsed'}, {'id': 150, 'status': 'Parsed'}]
(<sqlalchemy.ext.automap.platforms object at 0x7f24c0308d68>, 2)


Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


[{'id': 1, 'status': 'Parsed'}, {'id': 2, 'status': 'Parsed'}, {'id': 3, 'status': 'Parsed'}, {'id': 4, 'status': 'Parsed'}, {'id': 5, 'status': 'Parsed'}, {'id': 37, 'status': 'Parsed'}, {'id': 6, 'status': 'Parsed'}, {'id': 7, 'status': 'Parsed'}, {'id': 8, 'status': 'Parsed'}, {'id': 9, 'status': 'Parsed'}, {'id': 10, 'status': 'Parsed'}, {'id': 71, 'status': 'Parsed'}, {'id': 11, 'status': 'Parsed'}, {'id': 12, 'status': 'Parsed'}, {'id': 13, 'status': 'Parsed'}, {'id': 14, 'status': 'Parsed'}, {'id': 15, 'status': 'Parsed'}, {'id': 16, 'status': 'Parsed'}, {'id': 17, 'status': 'Parsed'}, {'id': 18, 'status': 'Parsed'}]


,raw_id,brand_id,keyword_id,instagram_id,instagram_user_id,instagram_user_name,instagram_followers_count,instagram_media_url,instagram_sentiment,instagram_text,instagram_lang,instagram_topics,instagram_url,created_at
0,1,1,1,2219511064557756639,177923314,None,None,https://scontent-cdt1-1.cdninstagram.com/v/t51...,Neutral or Undefined,InSpIrE tOi\nBut You can’t be me🧿\nA bon enten...,en,None,B7NSY3CA6Df,2020-01-12 05:19:18
1,2,1,2,2219509454289459427,1949336699,None,None,https://scontent-cdt1-1.cdninstagram.com/v/t51...,Neutral or Undefined,The End\n#winefriens #winelife #winetasting #w...,en,None,B7NSBbWi1jj,2020-01-12 05:16:06
2,3,1,1,2219510057874922077,8947635,None,None,https://scontent-cdt1-1.cdninstagram.com/v/t51...,Neutral or Undefined,Coup de cœur à la française. Voici le Bordeaux...,fr,None,B7NSKNfCppd,2020-01-12 05:17:18
3,4,1,2,2219509306541116013,222716366,None,None,https://scontent-cdt1-1.cdninstagram.com/v/t51...,Neutral or Undefined,梅酒のような味わい#france #vin #redwine #フランス #赤ワイン #vi...,ja,None,B7NR_RwDopt,2020-01-12 05:15:49
4,5,1,2,2219508715269812794,17763947591,None,None,https://scontent-cdt1-1.cdninstagram.com/v/t51...,Neutral or Undefined,Château margaux \n#wine #castle #chateau #vin,fr,None,B7NR2rFinI6,2020-01-12 05:14:38
5,37,1,2,2219484852281395616,5553574421,None,None,https://scontent-cdt1-1.cdninstagram.com/v/t51...,Neutral or Undefined,#vin #igstory \n#Asap,ht,None,B7NMba8l8mg,2020-01-12 04:27:18
6,6,1,1,2219509924556352194,6342802007,None,None,https://scontent-cdt1-1.cdninstagram.com/v/t51...,Neutral or Undefined,Le pavé de veau à l'émental \n#artisanboucher ...,fr,None,B7NSIRUoqrC,2020-01-12 05:17:02
7,7,1,2,2219505126842678565,4051773773,None,None,https://scontent-cdt1-1.cdninstagram.com/v/t51...,Neutral or Undefined,Corra Wine grape berries getting one last run ...,en,None,B7NRCdGnb0l,2020-01-12 05:14:23
8,8,1,1,2219509487156042643,186419638,None,None,https://scontent-cdt1-1.cdninstagram.com/v/t51...,Neutral or Undefined,Pause douceur ☕️ #chocolatchaud #douceur #bord...,fr,None,B7NSB59i5uT,2020-01-12 05:16:10
9,9,1,2,2219507025846872845,13988698596,None,None,https://scontent-cdt1-1.cdninstagram.com/v/t51...,Neutral or Undefined,‼️划重点了 快本播出时间于下周日晚八时20分\n\n小编已经等不及了 预告看了100遍了✨...,zh,None,B7NReFsFisN,2020-01-12 05:11:17


In [ ]:
           #df1 = (pd.concat({i: pd.io.json.json_normalize(x) for i, x in df.pop('raw_data').items()})
                 #.reset_index(level=1, drop=True)
                 #.join(df, rsuffix='raw',)
                 #.reset_index(drop=True))

In [3]:
        isReliable, textBytesFound, details = cld2.detect(twitter_text)
        twitter_lang = details[0][1])

fr
